#### <center>Intermediate Python and Software Enginnering</center>


## <center>Section 08 - Part 08 - Cython </center>


### <center>Innovation Scholars Programme</center>
### <center>King's College London, Medical Research Council and UKRI <center>

## Cython

* A separate programming language combining Python syntax (mostly) with static typing and C/C++ coding
* Used to write many wrapper libraries around C/C++/etc. code, eg. Numpy
* A bit complex to use with static typing, compilation, and other details
* Accessible through Jupyter with an extension:

In [ ]:
%load_ext cython

We then define code using Cython syntax:

In [29]:
%%cython

def fib_cython(int n):
    cdef int i, a, b # type declaration
    a, b = 1, 1
    for i in range(n):
        a, b = a + b, a
    return a

<IPython.core.display.Javascript object>

Compare against the pure Python code:

In [30]:
def fib(n):
    a, b = 1, 1
    for i in range(n):
        a, b = a + b, a
    return a


%timeit fib(20)
%timeit fib_cython(20)

777 ns ± 4.22 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
46.4 ns ± 0.811 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


<IPython.core.display.Javascript object>

* Need to declare variables with static type
* Those without type by default are Python objects
* Code using Python objects needs to call into CPython API so speed-up is mostly through definitions that can be pure C/C++
* Choose arguments and return values that have equivalent primitive values in C/C++ and Numpy arrays which Cython knows about

* We can "cythonize" regular .py files to get some speed-up:

In [16]:
%%writefile primes.py

def primes(nb_primes):
    p = []
    n = 2
    while len(p) < nb_primes:
        # Is n prime?
        if not any(n%i==0 for i in p):
            p.append(n)
        n += 1
    return p

Overwriting primes.py


<IPython.core.display.Javascript object>

In [17]:
!cp primes.py primes_cython.py
!cythonize -3 -i primes_cython.py > /dev/null
!ls -l

total 3604
-rw-r--r-- 1 localek10 bioeng    8868 Mar 18 15:35 01_exercises.ipynb
-rw-r--r-- 1 localek10 bioeng   57698 Mar 18 14:46 01_lecture.ipynb
-rw-r--r-- 1 localek10 bioeng   10404 Mar 18 15:37 01_solutions.ipynb
-rw-r--r-- 1 localek10 bioeng 1332381 Mar 18 21:03 02_exercises.ipynb
-rw-r--r-- 1 localek10 bioeng   56108 Mar 18 21:08 02_lecture.ipynb
-rw-r--r-- 1 localek10 bioeng 1662578 Mar 18 12:05 02_solutions.ipynb
-rw-r--r-- 1 localek10 bioeng     243 Mar 18 14:45 arraytest.py
-rw-r--r-- 1 localek10 bioeng     276 Mar 18 14:44 mptest.py
-rw-r--r-- 1 localek10 bioeng     389 Mar 18 14:45 pooltest.py
-rw-r--r-- 1 localek10 bioeng  218299 Mar 18 21:08 primes_cython.c
-rwxr-xr-x 1 localek10 bioeng  299584 Mar 18 21:08 primes_cython.cpython-37m-x86_64-linux-gnu.so
-rw-r--r-- 1 localek10 bioeng     188 Mar 18 21:08 primes_cython.py
-rw-r--r-- 1 localek10 bioeng     188 Mar 18 21:08 primes.py
drwxr-xr-x 2 localek10 bioeng    4096 Mar 18 20:35 __pycache__


<IPython.core.display.Javascript object>

In [18]:
import primes
import primes_cython

%timeit primes.primes(1000)
%timeit primes_cython.primes(1000)

33.5 ms ± 1.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
10.3 ms ± 6.92 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


<IPython.core.display.Javascript object>

* Cheap and easy way to optimize code
* Compiled C/C++ which calls into the CPython API
* Writing in Cython with explicit types will be much faster however

* Cython classes can be declared with `cdef`:

In [19]:
%%cython 

cdef class Point2:
    cdef readonly float x, y
    
    def __init__(self,float x, float y):
        self.x = x
        self.y = y
        
    def length(self):
        return (self.x**2+self.y**2)**0.5

<IPython.core.display.Javascript object>

* We get the usual speed-up with this code:

In [20]:
p = Point2(4, 5)
print(dir(p))
%timeit (p.x**2+p.y**2)**0.5
%timeit p.length()

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', 'length', 'x', 'y']
226 ns ± 16.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
87.8 ns ± 0.0137 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


<IPython.core.display.Javascript object>

* Inheritance still works:

In [21]:
class Point3(Point2):
    def __init__(self, x, y, z):
        super().__init__(x, y)
        self.z = z

    def length(self):
        return (self.x ** 2 + self.y ** 2 + self.z ** 2) ** 0.5


p3 = Point3(4, 5, 6)
print(p3.length())

8.774964387392123


<IPython.core.display.Javascript object>

* Cython classes don't have the same state as regular Python classes since they are implemented in C/C++ under the hood
* Advantage is speed, access to C++ types, interfacing with your own C++ types
* To interface with existing C++ code, a separate .pxd definition file must be created
* That can be used in a .pyx file to import C++ types and use this in Cython code

* Many more techniques to optimize code with Cython
* Using C++ data types can be faster, such as `std::vector`
* Writing the most important parts in pure C and interfacing through Cython
* Numpy arrays can also be used through Cython, providing access to large scale memory